In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22685415
paper_name = 'north_eide_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
files = ['journal.pgen.1002699.s002.xlsx','journal.pgen.1002699.s003.xlsx','journal.pgen.1002699.s004.xlsx']

In [12]:
original_data_list = []
for ixf, f in enumerate(files):
    original_data = pd.read_excel('raw_data/' + f, sheet_name='Table 1', skiprows=1)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    original_data['orf'] = original_data['ORF'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data.drop(columns =['Gene'], inplace=True)
    original_data.set_index('orf', inplace=True)
    
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 433 x 5
Empty DataFrame
Columns: [ORF, Gene, 5 generations, 15 generations, Unnamed: 4, orf]
Index: []
(433, 3)
Original data dimensions: 513 x 5
Empty DataFrame
Columns: [ORF, Gene, 5 generations, 15 generations, Unnamed: 4, orf]
Index: []
(513, 3)
Original data dimensions: 3663 x 3
Empty DataFrame
Columns: [ORF, Gene, Unnamed: 2, orf]
Index: []
(3642, 1)


In [13]:
original_data = pd.concat(original_data_list, axis=0)

In [14]:
original_data.head()

,5 generations,15 generations,Unnamed: 4,Unnamed: 2
orf,,,,
YAL009W,NaN,-0.8,NaN,NaN
YAL013W,-1.6,NaN,NaN,NaN
YAL040C,-1.4,-2.7,NaN,NaN
YAL055W,NaN,-2.9,NaN,NaN
YBL031W,NaN,-1.9,NaN,NaN


In [15]:
original_data.shape

(4588, 4)

In [18]:
original_data.drop(columns = ['Unnamed: 4', 'Unnamed: 2'], inplace=True)

In [19]:
original_data[original_data.isnull()] = 0

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [16282, 16283]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,16282,16283
data_type,value,value
orf,,
YAL009W,0.0,-0.8
YAL013W,-1.6,0.0
YAL040C,-1.4,-2.7
YAL055W,0.0,-2.9
YBL031W,0.0,-1.9


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16282,16283
,data_type,value,value
gene_id,orf,,
7,YAL009W,0.0,-0.8
11,YAL013W,-1.6,0.0
38,YAL040C,-1.4,-2.7
51,YAL055W,0.0,-2.9
119,YBL031W,0.0,-1.9


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16282,16283,16282,16283
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.0,0.0,-0.083172,-0.033770
4,YAL005C,0.0,0.0,-0.083172,-0.033770
5,YAL007C,0.0,0.0,-0.083172,-0.033770
6,YAL008W,0.0,0.0,-0.083172,-0.033770
7,YAL009W,0.0,-0.8,-0.083172,-0.988985


# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 22685415...
Inserting the new data...


100%|██████████| 2/2 [00:11<00:00,  5.77s/it]

Updating the data_modified_on field...
